<a href="https://colab.research.google.com/github/Zoro1092000/ZOZO/blob/master/Essemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Pip & Import.

In [4]:
import os
import os.path as osp
import torch
import sys

os.environ['TORCH'] = torch.__version__

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric==1.4.3
!pip install torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install deepdish==0.3.5

import torch.nn as nn
from torch.nn import Parameter
from torch_geometric.utils import scatter_
from torch_geometric.nn.inits import glorot, zeros
from torch_scatter import scatter_add
from torch.utils.data import DataLoader, Dataset


from itertools import chain
import pickle
import h5py
import deepdish as dd
import numpy as np
from tqdm import tqdm
import inspect
import time
import math
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# II. Data

In [5]:
# Data utils
def h5group_to_dict(h5group):
    group_dict = {k: v[()] for k, v in chain(h5group.items(), h5group.attrs.items())}
    return group_dict

def sub_dict(full_dict, *keys, to_tensor):
    return {k: torch.tensor(full_dict[k]) if to_tensor else full_dict[k] for k in keys if k in full_dict}

def build_graph_from_dict_pyg(graph_dict, to_tensor=True):
    from torch_geometric.data import Data

    g = Data(**sub_dict(graph_dict, 'edge_index', 'x', 'y', 'edge_attr', 'edge_y', to_tensor=to_tensor))
    return g

# Data loader
class GraphDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=128, shuffle=False, num_workers=0):

        def collate_graph(graph_obj_list):
            from torch_geometric.data import Batch
            batch = Batch.from_data_list(graph_obj_list)
            return batch

        super().__init__(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=collate_graph,
            num_workers=num_workers)

# BotnetDataset
class BotnetDataset(Dataset):

    def __init__(self, name='chord', root='data/botnet', split='train', graph_format='pyg', split_idx=None, add_nfeat_ones=True,
                 in_memory=True):
        super().__init__()
        assert name in ['chord', 'debru', 'kadem', 'leet', 'c2', 'p2p']
        assert split in ['train', 'val', 'test']

        if isinstance(root, str):
            root = osp.expanduser(osp.normpath(root))

        self.name = name
        self.root = root
        self.split = split
        self.split_idx = split_idx
        self.add_nfeat_ones = add_nfeat_ones

        self._graph_format = graph_format
        if split == 'train':
            self.path = self.processed_paths[0]
            self.num_graphs = 768
        elif split == 'val':
            self.path = self.processed_paths[1]
            self.num_graphs = 96
        elif split == 'test':
            self.path = self.processed_paths[2]
            self.num_graphs = 96

        # in_memory = False
        # self.data = h5py.File(self.path, 'r')
        self.data = None    # defer opening file in each process to make multiprocessing work
        self.data_type = 'file'
            
    @property
    def processed_dir(self):
        return osp.join(self.root, 'processed')

    @property
    def processed_file_names(self):
        return [self.name + '_' + s + '.hdf5' for s in ('train', 'val', 'test')]

    @property
    def processed_paths(self):
        return [osp.join(self.processed_dir, f) for f in self.processed_file_names]

    def __len__(self):
        return self.num_graphs

    def __getitem__(self, index):
        if self.data_type == 'dict':
            graph_dict = self.data[str(index)]
        elif self.data_type == 'file':
            if self.data is None:
                # only open once in each process
                self.data = h5py.File(self.path, 'r')
            graph_dict = h5group_to_dict(self.data[str(index)])
        else:
            raise ValueError

        # graph_format == 'pyg':
        return build_graph_from_dict_pyg(graph_dict)


    def __iter__(self):
        for i in range(self.num_graphs):
            yield self[i]

    def __repr__(self):
        return f'{self.__class__.__name__}(topology: {self.name} | split: {self.split} | ' \
               f'#graphs: {len(self)} | graph format: {self.graph_format})'


# III. Measure Performancce

## 3.1. Metrics

In [6]:
def f1(target, pred, label):
    # F1 = 2 * (precision * recall) / (precision + recall)
    tp = np.sum((target==label) & (pred==label))
    fp = np.sum((target!=label) & (pred==label))
    fn = np.sum((pred!=label) & (target==label))
    
    if tp+fp==0 or tp+fn==0:
      return np.nan

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    if precision+recall==0:
      return np.nan
      
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def f1_macro(pred, target):
    return np.mean([f1(target, pred, label) for label in range(0, 2)])


def accuracy(pred, target):
    return (pred == target).sum().item() / len(target)


def true_positive(pred, target):
    return (target[pred == 1] == 1).sum().item()


def false_positive(pred, target):
    return (target[pred == 1] == 0).sum().item()


def true_negative(pred, target):
    return (target[pred == 0] == 0).sum().item()


def false_negative(pred, target):
    return (target[pred == 0] == 1).sum().item()


def recall(pred, target):
    try:
        return true_positive(pred, target) / (target == 1).sum().item()
    except:  # divide by zero
        return -1


def precision(pred, target):
    try:
        prec = true_positive(pred, target) / (pred == 1).sum().item()
        return prec
    except:  # divide by zero
        return -1


def f1_score(pred, target):
    prec = precision(pred, target)
    rec = recall(pred, target)
    try:
        return 2 * (prec * rec) / (prec + rec)
    except:
        return 0


def false_positive_rate(pred, target):
    try:
        return false_positive(pred, target) / (target == 0).sum().item()
    except:  # divide by zero
        return -1


def false_negative_rate(pred, target):
    try:
        return false_negative(pred, target) / (target == 1).sum().item()
    except:  # divide by zero
        return -1


## 3.2. Evaluation

In [7]:
def eval_metrics(target, pred_prob, threshold=0.5):
    if isinstance(target, torch.Tensor):
        target = target.cpu().numpy()
    if isinstance(pred_prob, torch.Tensor):
        pred_prob = pred_prob.cpu().numpy()

    pred = (pred_prob >= threshold).astype(int)

    acc = accuracy(pred, target)
    fpr = false_positive_rate(pred, target)
    fnr = false_negative_rate(pred, target)
    rec = recall(pred, target)
    prc = precision(pred, target)
    f1 = f1_score(pred, target)
    f1macro = f1_macro(pred, target)
    result_dict = {'acc': acc, 'fpr': fpr, 'fnr': fnr, 'rec': rec, 'prc': prc, 'f1': f1, 'f1_macro': f1macro}

    return result_dict


def dict_value_add(dict1, dict2):
    result = {key: dict1.get(key, 0) + dict2.get(key, 0)
              for key in set(dict1) | set(dict2)}
    return result


def dict_value_div(dict, n):
    result = {key: value / n for key, value in dict.items()}
    return result


def eval_predictor(dataset, predictor):
    result_dict_avg = {}
    loss_avg = 0

    for data in dataset:
        # prediction
        try:
            pred_prob, loss = predictor(data)
            loss_avg += loss
        except ValueError:  # if "too many values to unpack"
            pred_prob = predictor(data)

        # get the ground truth target
        # graph_format == 'pyg':
        target = data.y

        # compute the evaluation metrics
        result_dict = eval_metrics(target, pred_prob)

        result_dict_avg = dict_value_add(result_dict_avg, result_dict)

    # average the metrics across all graphs in the dataset as final results
    result_dict_avg = dict_value_div(result_dict_avg, len(dataset))
    loss_avg = loss_avg / len(dataset)

    return result_dict_avg, loss_avg


# =================================================================================================================
# some examples of the 'predictor' model wrapper to be fed into the above evaluation function (for PyG Data format)
# =================================================================================================================
class PygRandomPredictor:
    def __init__(self):
        # torch.manual_seed(0)
        pass

    def __call__(self, data):
        pred_prob = torch.rand(len(data.y))
        return pred_prob


class PygModelPredictor:
    def __init__(self, model, loss_fcn=torch.nn.CrossEntropyLoss()):
        self.model = model
        self.loss_fcn = loss_fcn
        self.device = next(model.parameters()).device

    def __call__(self, data):
        self.model.eval()
        data = data.to(self.device)
        with torch.no_grad():
            # custom the below line to adjust to your model's input format for forward pass
            out = self.model(data.x, data.edge_index)
            loss = self.loss_fcn(out, data.y.long())
            pred_prob = torch.softmax(out, dim=1)[:, 1]
        return pred_prob, loss.float()


# IV. Model

## 5.1. Activation 

In [8]:
def activation(act, negative_slope=0.2):
    activations = nn.ModuleDict([
        ['lrelu', nn.LeakyReLU(negative_slope)],
        ['relu', nn.ReLU()],
        ['elu', nn.ELU()],
        ['none', nn.Identity()],
    ])
    return activations[act]

## 5.2. GIN model

In [9]:
import torch.nn as nn

class GINModel(nn.Module):
    def __init__(self, dim_input_feature, dim_hidden_feature, num_layers, num_classes, non_linear='relu', dropout=0.0):
        super().__init__()

        self.dim_input_feature = dim_input_feature
        self.dim_hidden_feature = dim_hidden_feature

        self.num_layers = num_layers
        self.num_classes = num_classes
        
        from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
        from torch_geometric.nn import GINConv
        
        self.gin_net = nn.ModuleList()
        for i in range(self.num_layers):
          if i == 0:
            self.gin_net.append(
                GINConv(
                  Sequential(Linear(self.dim_input_feature, self.dim_hidden_feature),
                             BatchNorm1d(self.dim_hidden_feature), ReLU(),
                             Linear(self.dim_hidden_feature, self.dim_hidden_feature), ReLU()), train_eps = True))
          else:
            self.gin_net.append(
                GINConv(
                  Sequential(Linear(self.dim_hidden_feature, self.dim_hidden_feature),
                             BatchNorm1d(self.dim_hidden_feature), ReLU(),
                             Linear(self.dim_hidden_feature, self.dim_hidden_feature), ReLU()), train_eps = True)) 

        self.dropout = nn.Dropout(dropout)

        self.residuals = nn.ModuleList()
        for i in range(self.num_layers):
          if i == 0:
            self.residuals.append(nn.Linear(self.dim_input_feature, self.dim_hidden_feature, bias=True))
          else:
            self.residuals.append(nn.Identity())

        self.num_residuals = len(self.residuals)

        self.non_linear = activation(non_linear)

        # self.final_type == 'proj':
        self.final = nn.Linear(self.dim_hidden_feature, num_classes)

    def reset_parameters(self):
        for net in self.gin_net:
            net.reset_parameters()
        # self.residual_hop = 1
        for net in self.residuals:
            net.reset_parameters()
        # self.final_type != 'none':
        self.final.reset_parameters()

    def forward(self, x, edge_index):
        xr = None
        add_xr_at = -1

        for n, net in enumerate(self.gin_net):
            # pass to a GIN layer with non-linear activation
            xo = net(x, edge_index)
            xo = self.dropout(xo)
            # deal with residual connections
            # self.residual_hop = 1
            if n < self.num_residuals:
                xr = self.residuals[n](x)
                add_xr_at = n
            if n == add_xr_at:
                xo = self.non_linear(xo + xr)

            x = xo
        # size of x: (B * N, dim_hidden_feature) -> (B * N, num_classes)
        x = self.final(x)

        return x


# V. Load data

In [10]:
data_dir = '/content/drive/Shareddrives/botnetdata/P2P'
data_name = 'p2p' # 'chord', 'debru', 'kadem', 'leet', 'c2', 'p2p'
batch_size = 8
in_memory = False
shuffle = False

# ========== load the dataset
print('loading dataset...')

train_ds = BotnetDataset(name=data_name, root=data_dir, split='train',
                         in_memory=bool(in_memory), graph_format='pyg')
val_ds = BotnetDataset(name=data_name, root=data_dir, split='val',
                       in_memory=bool(in_memory), graph_format='pyg')
test_ds = BotnetDataset(name=data_name, root=data_dir, split='test',
                        in_memory=bool(in_memory), graph_format='pyg')
train_loader = GraphDataLoader(train_ds, batch_size=batch_size, shuffle=bool(shuffle), num_workers=0)
print(data_name)

loading dataset...
p2p


# VII. Essemble

In [11]:
GCN = torch.load("/content/saved_models/GCN_model.pt")
GIN = torch.load("/content/saved_models/GIN_model.pt")
print(GCN)
print(GIN)

GCNModel(
  (gcn_net): ModuleList(
    (0): GCNLayer(
      (gcn): NodeModelAdditive (in_channels: 1, out_channels: 32, in_edgedim: None, deg_norm: rw, edge_gate: NoneType,aggr: add | number of parameters: 64)
      (non_linear): Identity()
    )
    (1): GCNLayer(
      (gcn): NodeModelAdditive (in_channels: 32, out_channels: 32, in_edgedim: None, deg_norm: rw, edge_gate: NoneType,aggr: add | number of parameters: 1056)
      (non_linear): Identity()
    )
    (2): GCNLayer(
      (gcn): NodeModelAdditive (in_channels: 32, out_channels: 32, in_edgedim: None, deg_norm: rw, edge_gate: NoneType,aggr: add | number of parameters: 1056)
      (non_linear): Identity()
    )
    (3): GCNLayer(
      (gcn): NodeModelAdditive (in_channels: 32, out_channels: 32, in_edgedim: None, deg_norm: rw, edge_gate: NoneType,aggr: add | number of parameters: 1056)
      (non_linear): Identity()
    )
    (4): GCNLayer(
      (gcn): NodeModelAdditive (in_channels: 32, out_channels: 32, in_edgedim: None, deg_

In [12]:
class PygModelPredictorEssemble:
    def __init__(self, modelGIN, modelGCN, loss_fcn=torch.nn.CrossEntropyLoss()):
        self.modelGIN = modelGIN
        self.modelGCN = modelGCN
        self.loss_fcn = loss_fcn
        self.device = next(modelGIN.parameters()).device
        self.device = next(modelGCN.parameters()).device

    def __call__(self, data):
        self.modelGIN.eval()
        self.modelGCN.eval()
        data = data.to(self.device)
        with torch.no_grad():
            # custom the below line to adjust to your model's input format for forward pass
            outGIN = self.modelGIN(data.x, data.edge_index)
            outGCN = self.modelGCN(data.x, data.edge_index)
            loss = self.loss_fcn((4*outGIN+outGCN)/5, data.y.long())
            pred_prob = torch.softmax((4*outGIN+outGCN)/5, dim=1)[:, 1]
        return pred_prob, loss.float()

In [13]:
predictorGCN = PygModelPredictor(GCN)

In [14]:
result_dict_avg_GCN, loss_avg_GCN = eval_predictor(test_ds, predictorGCN)
print(f'Testing --- loss: {loss_avg_GCN:.5f}')
print(' ' * 10 + ', '.join(['{}: {:.5f}'.format(k, v) for k, v in result_dict_avg_GCN.items()]))

Testing --- loss: 0.00781
          fnr: 0.01073, acc: 0.99886, f1: 0.97375, prc: 0.95912, fpr: 0.00093, rec: 0.98927, f1_macro: 0.98658


In [15]:
predictorGIN = PygModelPredictor(GIN)

In [16]:
result_dict_avg_GIN, loss_avg_GIN = eval_predictor(test_ds, predictorGIN)
print(f'Testing --- loss: {loss_avg_GIN:.5f}')
print(' ' * 10 + ', '.join(['{}: {:.5f}'.format(k, v) for k, v in result_dict_avg_GIN.items()]))

Testing --- loss: 0.00136
          fnr: 0.00382, acc: 0.99980, f1: 0.99524, prc: 0.99480, fpr: 0.00012, rec: 0.99618, f1_macro: 0.99757


In [17]:
predictorEssemble = PygModelPredictorEssemble(GIN, GCN)

In [18]:
result_dict_avg_Essemble, loss_avg_Essemble = eval_predictor(test_ds, predictorEssemble)
print(f'Testing --- loss: {loss_avg_Essemble:.5f}')
print(' ' * 10 + ', '.join(['{}: {:.5f}'.format(k, v) for k, v in result_dict_avg_Essemble.items()]))

Testing --- loss: 0.00147
          fnr: 0.00378, acc: 0.99981, f1: 0.99531, prc: 0.99491, fpr: 0.00012, rec: 0.99622, f1_macro: 0.99761
